In [1]:
# Librerias utiles
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import plotly
#import seaborn as sns 
import plotly.express as px
import missingno as msno
import plotly.io as pio
import plotly.graph_objects as go
from mapsmx import MapsMX
import geopandas
import pyproj
import json
#sns.set_theme()

In [2]:
state = MapsMX().get_geo('state')
muns = MapsMX().get_geo('municipality')
yuc = muns[muns['cve_ent']=='31']

In [3]:
victimas = pd.read_csv('reportes/reporte_semujeres_11_OCTUBRE_2021.csv',low_memory=False, dtype={'pk_perfil_agresor': 'object','num_hijos':'int32'}, parse_dates=['fecha_recepcion','fecha_hechos'])

In [ ]:
victimas_2020 = victimas[victimas.año_recepcion==2020]

In [ ]:
victimas_2019 = victimas[victimas.año_recepcion==2019]

In [ ]:
# censo poblacion y vivienda 2020 inegi
inegi_2020 = pd.read_csv('censo_yuc.csv')
pob_fem = inegi_2020[inegi_2020.NOM_LOC=='Total del Municipio'][['MUN','NOM_MUN','POBFEM' ]].reset_index(drop=True)

In [ ]:
casos_2020 = victimas_2020.groupby('municipiohechos')['fk_euv'].count()
casos_2020 = yuc[['geometry_mun','nom_mun']].merge(casos_2020, right_on=casos_2020.index, left_on='nom_mun', how='left')
casos_2020.fillna(0,inplace=True)
casos_2020['fk_euv']=casos_2020['fk_euv'].astype(int)
casos_2020 = casos_2020.rename(columns={'fk_euv':'casos_2020'})

In [ ]:
casos_2019 = victimas_2019.groupby('municipiohechos')['fk_euv'].count()
casos_2019 = yuc[['geometry_mun','nom_mun']].merge(casos_2019, right_on=casos_2019.index, left_on='nom_mun', how='left')
casos_2019.fillna(0,inplace=True)
casos_2019['fk_euv']=casos_2019['fk_euv'].astype(int)
casos_2019 = casos_2019.rename(columns={'fk_euv':'casos_2019'})

In [ ]:
dif_casos = casos_2020.merge(casos_2019[['nom_mun','casos_2019']], left_on='nom_mun', right_on='nom_mun', how='left')

In [ ]:
dif_casos

In [ ]:
var = round(dif_casos[['casos_2019','casos_2020']].pct_change(axis='columns')*100,2)
dif_casos = dif_casos.merge(var['casos_2020'], left_on=dif_casos.index, right_on=var.index, how='left')
dif_casos.drop(columns=['key_0'], inplace=True)

In [ ]:
dif_casos = dif_casos.rename(columns={'casos_2020_y':'Variación'})

In [ ]:
dif_casos

In [ ]:
total_casos=dif_casos.casos_2020_x.sum()+dif_casos.casos_2019.sum()
total_casos

In [ ]:
gdf = geopandas.GeoDataFrame(dif_casos, geometry='geometry_mun')
gdf = gdf.set_index('nom_mun')
gdf = gdf.to_crs(pyproj.CRS.from_epsg(4326))

In [ ]:
inf = gdf[gdf.Variación.isin([np.inf])].index
gdf.loc[inf,'Variación']=gdf.loc[inf,'casos_2020_x']*100

In [ ]:
gdf['total'] = gdf['casos_2020_x'] + gdf['casos_2019']

In [ ]:
gdf

In [ ]:
fig = px.choropleth_mapbox(gdf, geojson=gdf.geometry_mun, 
                    locations=gdf.index, color="Variación",
                    color_continuous_scale="RdBu_r",
                    labels={'nom_mun': 'Municipio',"casos_2020_x":'Casos 2020','casos_2019':'Casos 2019' },
                    #color_continuous_midpoint=np.average(gdf['Variación']),
                    #color_continuous_midpoint=0,
                    #labels = {'nom_mun':'Municipio', 'municipioV':'Víctimas','EdadVictima':'Edades','tipos':'Violencias','año_recepcion':'Años','TipoRelacion':'Relación','pk_perfil_agresor':'pk agresor'},
                    hover_data={"casos_2020_x":True,'casos_2019':True },
                    title='Mapa 5. Variación porcentual de casos registrados',
                    #color_continuous_midpoint=np.average(victimas['Edad']),
                    center={"lat": 20.400417, "lon": -89.134857},
                    mapbox_style="carto-positron", 
                    zoom=7.5,
                    #opacity=0.8,
                    )
fig.update_geos(fitbounds="locations", visible=False)

#fig.update_layout(xaxis=dict(domain=[0, 0.5]), yaxis=dict(domain=[0.25, 0.75]))

fig.add_annotation(
    # The arrow head will be 25% along the x axis, starting from the left
    x=0,
    # The arrow head will be 40% along the y axis, starting from the bottom
    y=0.98,
    text="<b>Temporalidad:</b> {}-{}<br><b>Total de casos:</b> {}".format(2019,2020,total_casos),
    showarrow=False,
    bordercolor="black",
    bgcolor="white",
    borderwidth=1.5,
    opacity=0.8
)

fig.update(layout = dict(title=dict(x=0.5)))
fig.update_layout(
    margin={"r":0,"t":30,"l":10,"b":10},
    coloraxis_colorbar={
        'title':'Variación',}, 
    coloraxis_colorbar_ticksuffix='%'),
fig.update_layout(title_y=1, title_x=0)
fig.update_layout(coloraxis_showscale=True)
fig.update_coloraxes(cmax=100, cmin=-100)

#print("plotly express hovertemplate:", fig.data[0].hovertemplate)
fig.update_traces(hovertemplate='Municipio=%{location}<br>Casos 2020=%{customdata[0]}<br>Casos 2019=%{customdata[1]}<br>Variación=%{z}%<extra></extra>')
 
pio.write_html(fig, file='../docs/figs/indicador_municipal_mapa_variacion_casos.html', include_plotlyjs='cdn', full_html=False)
fig.show()